## Used for selecting the 6 best features per cluster
* We're using mean squared error of each variable vs. the ALSFRS_score, and take the best 6. 

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from IPython.display import display

from modeling_funcs import *


In [2]:
vectorized_data = pd.read_csv('../train_data_vectorized.csv', sep='|', index_col=0)
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col=0)
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col=0)
all_feature_metadata = pickle.load( open('../all_feature_metadata.pickle', 'rb') )

X = vectorized_data.join(clusters)
Y = slope.join(clusters)
display(X.head())
display(Y.head())

,family_ALS_hist_last,weight_pct_diff,weight_mean_slope,weight_mean,weight_median,weight_std,F,M,Age_last,respiratory_rate_pct_diff,...,bp_diastolic_median,bp_diastolic_std,Creatinine_pct_diff,Creatinine_last,Creatinine_mean_slope,Creatinine_mean,Creatinine_median,Creatinine_std,fvc_percent_pct_diff,cluster
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,-0.232289,-0.002066,-1.620202,-1.617145,-0.435513,1.322796,-1.322796,0.941297,-0.169340,...,0.296494,0.345707,-0.025200,0.563661,0.175924,0.547815,0.558684,-1.276692,-1.864018,2
649,0,0.218932,0.466682,-0.086445,-0.100773,-0.163334,1.322796,-1.322796,-0.558473,0.169753,...,-0.162557,-1.053767,-0.023280,-0.863703,0.449074,-1.117773,-0.958224,-0.357095,-0.037980,0
1234,0,0.340632,0.124007,1.246644,1.224432,-0.325885,-0.755549,0.755549,-1.440691,0.658048,...,-0.621608,0.526932,-0.025200,1.039449,0.175924,1.060304,1.064321,-1.276692,0.993322,1
2492,0,-0.047759,-0.067460,0.924490,0.916621,-0.796321,-0.755549,0.755549,0.764854,-0.169340,...,0.411257,0.254585,-0.022054,0.087873,0.437748,-0.220918,-0.199770,0.023815,-0.361454,0
2956,0,-0.149486,-0.553742,-0.702551,-0.700194,-0.487343,1.322796,-1.322796,0.764854,-0.169340,...,-0.851133,0.127369,-0.023579,-0.387915,0.214460,-0.861529,-0.958224,-0.357095,0.661195,2


,ALSFRS_slope,cluster
SubjectID,,
533,-0.965608,2
649,-0.921717,0
1234,-0.914787,1
2492,-0.598361,0
2956,-0.444039,2


In [3]:
best_features_per_cluster = get_best_features_per_cluster(X, Y, all_feature_metadata)
best_features_per_cluster

{0: ['Age', 'BMI', 'Gender', 'family_ALS_hist', 'fvc_percent', 'height'],
 1: ['Age', 'BMI', 'Gender', 'Race', 'height', 'pulse'],
 2: ['Age', 'BMI', 'Gender', 'Race', 'bp_systolic', 'height']}

In [4]:
with open("../best_features_per_cluster.pickle", "wb") as output_file:
    pickle.dump(best_features_per_cluster, output_file)


#Apply the selector 
leave only the best features per cluster

In [5]:
for t in ["train", "test"]:
    print t
    df = pd.read_csv('../' + t + '_data.csv', sep = '|', index_col="SubjectID", dtype='unicode')
    print "df", df.shape
    clusters = pd.read_csv('../' + t + '_kmeans_clusters.csv', sep = '|', index_col="SubjectID")
    print "clusters", clusters.shape
    buf = filter_only_selected_features(df, clusters, best_features_per_cluster)
    with open('../' + t + '_data_selected.csv','w') as f:
        f.write(buf)


train
df (1138647, 5)
clusters (1777, 1)
0 (392660, 6) (8306, 6)
1 (337486, 6) (7262, 6)
2 (408501, 6) (8915, 6)
test
df (126664, 5)
clusters (600, 1)
0 (43544, 6) (1567, 6)
1 (33950, 6) (1312, 6)
2 (49170, 6) (1843, 6)
